### Use selected gene sets to predict age of holdout cells
- Using sets of selected genes at different sparsity levels to calculate prediction accuracy
- calculate accuracy with random genes sets of the same size

In [1]:
library(boot)
library(glmnet)
library(ggplot2)
library(dplyr)
library(caret)
library(gridExtra)

Loading required package: Matrix

Loading required package: foreach

Loaded glmnet 2.0-18



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: lattice


Attaching package: ‘lattice’


The following object is masked from ‘package:boot’:

    melanoma



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




In [3]:
options(stringsAsFactors = FALSE)

#### Prediction functions

In [4]:
predict_day_glmnet_v2 = function(training_mat, training_metadata, holdout_mat, holdout_metadata, genes.use, standardize_glmnet = TRUE, lambda = 0.000001, title = "selected_genes") {
    
    # fit model using training data
    training_mat = t(training_mat)

    small_mol_cells = training_metadata$protocol_final == "SM"
    training_mat = training_mat[small_mol_cells,, drop = FALSE]

    diff_day = training_metadata$day[small_mol_cells]

    # convert days to early/late
    diff_day = gsub('D12|D14', "early", diff_day)
    diff_day = gsub('D24|D26', "late", diff_day)
    names(diff_day) = training_metadata$X[small_mol_cells]
    diff_day = factor(diff_day, levels=c("early", "late"))
    #print(contrasts(diff_day))
    
    # subset to just genes in selected set
    subset_cardio = training_mat[,genes.use, drop = FALSE]
    subset_cardio = as.matrix(subset_cardio)
    
    # get gene-wise means and std devs from the training dataset
    ncol = ncol(subset_cardio)
    column_means = colMeans(subset_cardio)
    column_sds = apply(subset_cardio, 2, sd)
    
    # scale and center training data
    subset_cardio = scale(subset_cardio, scale = TRUE, center = TRUE)
    
    # add dummy all 0 variable whenever there is only 1 gene in matrix
    if (length(colnames(subset_cardio)) < 2) {
        dummy_var = rep(0, length(subset_cardio[, 1]))
        subset_cardio = cbind(subset_cardio, dummy_var)
    }
    # fit regression model with glmnet and ridge penalty (alpha = 0)
    fit = glmnet(subset_cardio, diff_day, family = "binomial", alpha = 0, lambda = lambda, standardize = standardize_glmnet)
    
    # save and plot coefficients
    fit_coeff = as.data.frame(as.matrix(coef(fit)))
    colnames(fit_coeff) = c("coeff")
    fit_coeff$gene = rownames(fit_coeff)
    write.csv(fit_coeff, file = paste0("coeffs_glmnet/", title, "_coeffs.csv"))
    
    p_coeff = ggplot(fit_coeff, aes(x = gene, y = coeff)) + geom_bar(stat = "identity") + theme(axis.text.x = element_text(angle = 50, hjust = 1))

    # predict time point in holdout data
    holdout_mat = t(holdout_mat)

    holdout_small_mol_cells = holdout_metadata$protocol_final == "SM"
    holdout_mat = holdout_mat[holdout_small_mol_cells,, drop = FALSE]

    holdout_diff_day = holdout_metadata$day[holdout_small_mol_cells]

    holdout_diff_day = gsub('D12|D14', "early", holdout_diff_day)
    holdout_diff_day = gsub('D24|D26', "late", holdout_diff_day)
    names(holdout_diff_day) = holdout_metadata$X[holdout_small_mol_cells]
    
    holdout_subset = holdout_mat[,genes.use, drop = FALSE]
    holdout_subset = as.matrix(holdout_subset)

    # instead of using scale to center and scale the holdout matrix, use scaling from trainin set
    gene_list = colnames(holdout_subset)
    holdout_subset = sapply(1:ncol,
                            FUN = function(x) {
                                current_col = holdout_subset[, x]
                                scaled_col = (current_col - column_means[x]) / column_sds[x]
                                return(scaled_col)
                            }
                           )
    colnames(holdout_subset) = gene_list    
    
    # add dummy all 0 variable whenever there is only 1 gene in matrix
    if (length(colnames(holdout_subset)) < 2) {
        dummy_var = rep(0, length(holdout_subset[, 1]))
        holdout_subset = cbind(holdout_subset, dummy_var)
    }
    
    holdout_predict = predict(fit, newx = holdout_subset, type = c("response"), s = lambda)

    if (identical(rownames(holdout_predict), names(holdout_diff_day))) {
        predicted_day = sapply(holdout_predict, get_predicted_time, simplify = TRUE)
        predict_actual_df = data.frame("predicted" = factor(predicted_day, levels = c("early", "late")), "actual" = factor(holdout_diff_day, levels = c("early", "late")))
    } else {
        stop("Cell names don't match")
    }

    mat_stats = caret::confusionMatrix(predict_actual_df$predicted, predict_actual_df$actual)

    # save confusion matrix as a plot
    p_confusion_mat = tableGrob(as.matrix(mat_stats$table))
    p_combined = grid.arrange(p_coeff, p_confusion_mat)
    ggsave(p_combined, filename = paste0("gene_set_predictions_glmnet/", title, "_coeffs_conf_mat.png"))

    stats_df = cbind(t(as.data.frame(mat_stats$byClass)), t(as.data.frame(mat_stats$overall)))
    # write.csv(stats_df, file = paste0(title, "_predict_stats.csv"))  # saving each one individually
    return(stats_df)
    
}

# late=1; early=0
get_predicted_time <- function(predict_prob) {
    if (predict_prob < 0.5) {
        return("early")
    } else {
        return("late")
    }
}

get_gene_name = function(name){
    gene_name = paste(name, "HUMAN", sep = "_")
    return(gene_name)
}

#### Load training, holdout, and bootstrapped penalizes regression results

In [25]:
base_dir = "/allen/aics/gene-editing/RNA_seq/scRNAseq_SeeligCollaboration/2019_analysis/feature_selection/"

Bootstrapped regression results for intermediate time points (12v24; 14v26)

In [26]:
load(paste0(base_dir, "results/early_late_small_mol.RData"))

In [27]:
lambda_sequence = bootstraps[[1]][["lambda"]]
genes = rownames(coef(bootstraps[[1]], s = lambda_sequence[1]))

Training and holdout cells and metadata

In [28]:
load("/allen/aics/gene-editing/RNA_seq/scRNAseq_SeeligCollaboration/2019_analysis/feature_selection/train_cm_entrez_only_normalized_counts.RData")
load("/allen/aics/gene-editing/RNA_seq/scRNAseq_SeeligCollaboration/2019_analysis/feature_selection/holdout_cm_entrez_only_normalized_counts.RData")

cell_metadata = read.csv("/allen/aics/gene-editing/RNA_seq/scRNAseq_SeeligCollaboration/2019_analysis/feature_selection/train_cm_entrez_only_metadata.csv")
holdout_cell_metadata = read.csv("/allen/aics/gene-editing/RNA_seq/scRNAseq_SeeligCollaboration/2019_analysis/feature_selection/holdout_cm_entrez_only_metadata.csv")

Checking cell and gene counts

In [10]:
table(cell_metadata$day)


 D12  D14  D24  D26 
4283 4228 2014 2290 

In [12]:
table(holdout_cell_metadata$day)


D12 D14 D24 D26 
481 476 227 259 

In [13]:
dim(cardio)

[1] 23665 12815

In [14]:
dim(holdout_cardio)

[1] 23665  1443

*** 
#### Make list of genes that are not all zero in both the training and holdout set
- for random sampling will filter only all 0 genes in training set; will not filter genes based on all zero in holdout

In [ ]:
# SM = protocol1
cardio_small_molecule_cells = cell_metadata$X[cell_metadata$protocol_final == "SM"]
holdout_small_molecule_cells = holdout_cell_metadata$X[holdout_cell_metadata$protocol_final == "SM"]
sm_only_cardio = cardio[, cardio_small_molecule_cells]
sm_only_holdout = holdout_cardio[, holdout_small_molecule_cells]

training_nozero_genes = rownames(sm_only_cardio[rowSums(sm_only_cardio) > 0, ])
holdout_nozero_genes = rownames(sm_only_holdout[rowSums(sm_only_holdout) > 0, ])

In [16]:
length(training_nozero_genes)
length(holdout_nozero_genes)

[1] 22324

[1] 18684

Will use `training_non_zero_genes` for random sampling (not filtering based on holdout data)

***
#### Get set of top 40 selected genes

In [18]:
selected_gene_df = read.csv("feature_selected_genes.csv", stringsAsFactors = FALSE)

In [19]:
selected_genes = selected_gene_df$X

***
### Cell age predictions accuracy in holdout cells using sets of selected genes at different levels of sparsity (different lambda penalty)
1. go through sequence of 100 lambdas and get selected genes (non-zero coefficient in all 1000 bootstrap rounds)
2. fit cell age ~ selected genes model with training data
3. use model fit on training data to predict cell age in holdout data
4. calculate accuracy of prediction in holdout data

In [ ]:
all_stats_df = data.frame()
current_lambda = lambda_sequence[2:100]  # skipping first lambda b/c there are 0 selected genes
for (i in 1:length(current_lambda)) {
    title = paste0("lambda", i + 1)

    nonzero_gene_count = c()
    lambda = current_lambda[i]

    genes_coeff = coef(bootstraps[[1]], s = lambda)
    genes_coeff = genes_coeff[2:length(genes_coeff),]
    gene_names = names(genes_coeff)
    genes_nonzero = gene_names[abs(genes_coeff) > 0]
    nonzero_gene_count = c(nonzero_gene_count, length(genes_nonzero))

    genes_intersect = genes_nonzero
    print(length(genes_intersect))

    for (i in 2:length(bootstraps)) {
        lambda_seq = bootstraps[[i]][["lambda"]]
        if (lambda < tail(lambda_seq, n = 1)) {
            print(i)
        } 
        genes_coeff = coef(bootstraps[[i]], s = lambda)
        genes_coeff = genes_coeff[2:length(genes_coeff),]
        gene_names = names(genes_coeff)
        genes_nonzero = gene_names[abs(genes_coeff) > 0]
        nonzero_gene_count = c(nonzero_gene_count, length(genes_nonzero))
        genes_intersect = intersect(genes_intersect, genes_nonzero)
    }
    print(length(genes_intersect))
    
    current_df = predict_day_glmnet_v2(training_mat = cardio,
                                    training_metadata = cell_metadata,
                                    holdout_mat = holdout_cardio,
                                    holdout_metadata = holdout_cell_metadata,
                                    genes.use = genes_intersect,
                                    standardize_glmnet = FALSE,
                                    lambda = 0.000001,
                                    title = title)

    rownames(current_df) = title
    current_df = as.data.frame(current_df)
    current_df$ngenes = length(genes_intersect)
    current_df$gene_type = "selection"

    all_stats_df = rbind(all_stats_df, current_df)
    
}

In [23]:
unique(all_stats_df)

,Sensitivity,Specificity,Pos Pred Value,Neg Pred Value,Precision,Recall,F1,Prevalence,Detection Rate,Detection Prevalence,Balanced Accuracy,Accuracy,Kappa,AccuracyLower,AccuracyUpper,AccuracyNull,AccuracyPValue,McnemarPValue,ngenes,gene_type
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
lambda2,0.8835759,0.6387665,0.8382643,0.7213930,0.8382643,0.8835759,0.8603239,0.6793785,0.6002825,0.7161017,0.7611712,0.8050847,0.5386315,0.7739521,0.8336493,0.6793785,4.871042e-14,0.033325368,1,selection
lambda3,0.9209979,0.6828194,0.8601942,0.8031088,0.8601942,0.9209979,0.8895582,0.6793785,0.6257062,0.7274011,0.8019087,0.8446328,0.6286796,0.8158147,0.8705416,0.6793785,7.831379e-24,0.001652788,2,selection
lambda5,0.9313929,0.7621145,0.8924303,0.8398058,0.8924303,0.9313929,0.9114954,0.6793785,0.6327684,0.7090395,0.8467537,0.8771186,0.7108712,0.8506478,0.9003954,0.6793785,1.409784e-34,0.032014857,3,selection
lambda11,0.9459459,0.8281938,0.9210526,0.8785047,0.9210526,0.9459459,0.9333333,0.6793785,0.6426554,0.6977401,0.8870699,0.9081921,0.7860251,0.8844795,0.9284304,0.6793785,1.150037e-47,0.136641005,6,selection
lambda14,0.9459459,0.8193833,0.9173387,0.8773585,0.9173387,0.9459459,0.9314227,0.6793785,0.6426554,0.7005650,0.8826646,0.9053672,0.7789191,0.8813780,0.9259081,0.6793785,2.413249e-46,0.087197064,7,selection
lambda15,0.9480249,0.8458150,0.9287169,0.8847926,0.9287169,0.9480249,0.9382716,0.6793785,0.6440678,0.6935028,0.8969200,0.9152542,0.8031821,0.8922600,0.9347088,0.6793785,4.232165e-51,0.245278117,9,selection
lambda16,0.9542620,0.8634361,0.9367347,0.8990826,0.9367347,0.9542620,0.9454171,0.6793785,0.6483051,0.6920904,0.9088490,0.9251412,0.8263485,0.9032239,0.9434259,0.6793785,3.061107e-56,0.271818443,10,selection
lambda18,0.9542620,0.8634361,0.9367347,0.8990826,0.9367347,0.9542620,0.9454171,0.6793785,0.6483051,0.6920904,0.9088490,0.9251412,0.8263485,0.9032239,0.9434259,0.6793785,3.061107e-56,0.271818443,11,selection
lambda20,0.9563410,0.8942731,0.9504132,0.9062500,0.9504132,0.9563410,0.9533679,0.6793785,0.6497175,0.6836158,0.9253070,0.9364407,0.8535926,0.9158740,0.9532652,0.6793785,1.202572e-62,0.765594484,12,selection


***
#### Add prediction based on set of D12/24 highly variable genes to all_stats_df

In [24]:
D12_D24_highly_var_genes = read.csv("../small_molecule_cardio_timepoints_indepth/D12_D24_highly_variable_genes.csv", stringsAsFactors = FALSE)

In [26]:
dim(D12_D24_highly_var_genes)

[1] 2241    1

In [27]:
D12_D24_highly_var_genes$name = sapply(D12_D24_highly_var_genes$var_gene, get_gene_name)

,var_gene,name
,<chr>,<chr>
1,MAN1C1,MAN1C1_HUMAN
2,ACTA1,ACTA1_HUMAN
3,CRB1,CRB1_HUMAN
4,FBN2,FBN2_HUMAN
5,NLGN1,NLGN1_HUMAN
6,FAM155A,FAM155A_HUMAN


Only use highly variable genes that were used in the bootsrapped feature selection analysis; we filtered out genes that did not have entrez gene ids when doing the feature selection analysis

In [33]:
highly_var_genes = intersect(D12_D24_highly_var_genes$name, training_nozero_genes)

In [34]:
length(highly_var_genes)

[1] 1877

In [35]:
length(D12_D24_highly_var_genes$name) - length(highly_var_genes)

[1] 364

364 genes from the highly variable list are not in this data set; there are a lot of RNA genes and some genes that likely had their gene symbols changed at some point

In [36]:
setdiff(D12_D24_highly_var_genes$name, training_nozero_genes)

[1] "RP11-410D17.2_HUMAN"  "AC010983.1_HUMAN"     "KAL1_HUMAN"          
  [4] "AC005614.5_HUMAN"     "RP11-120I21.2_HUMAN"  "RP11-475N22.4_HUMAN" 
  [7] "RP11-125B21.2_HUMAN"  "RP11-611E13.2_HUMAN"  "RP11-368L12.1_HUMAN" 
 [10] "RP11-118B18.1_HUMAN"  "AC097721.2_HUMAN"     "LPHN3_HUMAN"         
 [13] "RP11-420A23.1_HUMAN"  "RP11-405A12.2_HUMAN"  "AC004158.2_HUMAN"    
 [16] "AC092594.1_HUMAN"     "CTD-2215E18.1_HUMAN"  "RP13-188A5.1_HUMAN"  
 [19] "RP11-305L7.3_HUMAN"   "RP5-1103G7.4_HUMAN"   "RP11-166D19.1_HUMAN" 
 [22] "KIAA1462_HUMAN"       "RP11-85K15.2_HUMAN"   "LPHN2_HUMAN"         
 [25] "CTD-2337A12.1_HUMAN"  "RP11-69I8.3_HUMAN"    "LINC01021_HUMAN"     
 [28] "RP11-115D19.1_HUMAN"  "RP11-385J1.2_HUMAN"   "RP11-486A14.2_HUMAN" 
 [31] "RP11-313J2.1_HUMAN"   "RP11-563P16.1_HUMAN"  "TTC18_HUMAN"         
 [34] "RP11-141M1.3_HUMAN"   "RP11-665G4.1_HUMAN"   "RP3-410B11.1_HUMAN"  
 [37] "C20orf166_HUMAN"      "RP11-736K20.5_HUMAN"  "RP11-820L6.1_HUMAN"  
 [40] "PCNXL2_HUMAN"         "LRRC16A_HUMAN"        "RP11-123O10.4_HUMAN" 
 [43] "AC074363.1_HUMAN"     "RP11-131L23.1_HUMAN"  "AF131217.1_HUMAN"    
 [46] "RP11-440L14.1_HUMAN"  "LINC00478_HUMAN"      "RP11-130F10.1_HUMAN" 
 [49] "FAM65B_HUMAN"         "RP11-175E9.1_HUMAN"   "RP4-655J12.4_HUMAN"  
 [52] "AC010967.2_HUMAN"     "TM4SF2_HUMAN"         "RP11-155G15.2_HUMAN" 
 [55] "RP11-60L3.1_HUMAN"    "RP11-804N13.1_HUMAN"  "C1orf168_HUMAN"      
 [58] "RP4-769N13.6_HUMAN"   "AC097662.2_HUMAN"     "RP1-93I3.1_HUMAN"    
 [61] "RP11-499P20.2_HUMAN"  "FAM134B_HUMAN"        "LEPREL1_HUMAN"       
 [64] "WBSCR17_HUMAN"        "RP11-142I20.1_HUMAN"  "CTD-2147F2.1_HUMAN"  
 [67] "RP11-176N18.2_HUMAN"  "GPR126_HUMAN"         "AC007362.1_HUMAN"    
 [70] "RP1-78O14.1_HUMAN"    "FAM101A_HUMAN"        "AC009961.3_HUMAN"    
 [73] "PVRL3-AS1_HUMAN"      "CTC-340A15.2_HUMAN"   "RP11-356J5.12_HUMAN" 
 [76] "RP11-196E1.3_HUMAN"   "AC091969.1_HUMAN"     "KIRREL_HUMAN"        
 [79] "AC072062.1_HUMAN"     "RP11-187O7.3_HUMAN"   "RP11-328K4.1_HUMAN"  
 [82] "CTD-2143L24.1_HUMAN"  "RP11-260O18.1_HUMAN"  "AC027612.6_HUMAN"    
 [85] "AC005022.1_HUMAN"     "RP11-222A11.1_HUMAN"  "PTRF_HUMAN"          
 [88] "BAI3_HUMAN"           "RP3-399L15.3_HUMAN"   "LHFP_HUMAN"          
 [91] "RP11-1L12.3_HUMAN"    "AC074391.1_HUMAN"     "CTB-161M19.4_HUMAN"  
 [94] "RP11-844P9.2_HUMAN"   "C4orf21_HUMAN"        "RP11-120D5.1_HUMAN"  
 [97] "AC016629.8_HUMAN"     "RP11-499F3.2_HUMAN"   "RP11-22P4.1_HUMAN"   
[100] "DAK_HUMAN"            "AC026188.1_HUMAN"     "RP11-762E8.1_HUMAN"  
[103] "AC107218.3_HUMAN"     "RP11-427M20.1_HUMAN"  "CTD-2008L17.1_HUMAN" 
[106] "EFTUD1_HUMAN"         "KIAA0922_HUMAN"       "CTC-806A22.1_HUMAN"  
[109] "LINC00883_HUMAN"      "PPAP2A_HUMAN"         "C21orf49_HUMAN"      
[112] "KB-1047C11.2_HUMAN"   "RP11-163N6.2_HUMAN"   "RP11-144F15.1_HUMAN" 
[115] "RP11-371F15.3_HUMAN"  "RP11-114N19.3_HUMAN"  "RP11-444D3.1_HUMAN"  
[118] "GAREM_HUMAN"          "LINC01057_HUMAN"      "LARGE_HUMAN"         
[121] "SGK223_HUMAN"         "RP11-572M11.4_HUMAN"  "RP11-96H17.1_HUMAN"  
[124] "RP11-798K3.4_HUMAN"   "RP11-550P17.5_HUMAN"  "RP11-945C19.4_HUMAN" 
[127] "RP11-1E3.1_HUMAN"     "RP11-81N13.1_HUMAN"   "RP11-446H18.5_HUMAN" 
[130] "RP11-78F17.1_HUMAN"   "CTD-2260A17.2_HUMAN"  "LPPR4_HUMAN"         
[133] "AC092295.7_HUMAN"     "RP11-545I5.3_HUMAN"   "RP11-455B3.1_HUMAN"  
[136] "RP11-256I9.3_HUMAN"   "RP11-3B12.2_HUMAN"    "RP11-714G18.1_HUMAN" 
[139] "C10ORF68_HUMAN"       "AP001347.6_HUMAN"     "RP11-122D10.1_HUMAN" 
[142] "RP11-142A5.1_HUMAN"   "RP1-28O10.1_HUMAN"    "C10orf11_HUMAN"      
[145] "RP11-690J15.1_HUMAN"  "RP11-351M8.2_HUMAN"   "RP11-317J19.1_HUMAN" 
[148] "C4orf32_HUMAN"        "RP11-398K22.12_HUMAN" "AC034220.3_HUMAN"    
[151] "RP11-624L4.1_HUMAN"   "GPR98_HUMAN"          "RP1-34H18.1_HUMAN"   
[154] "CTD-2206N4.4_HUMAN"   "CTC-535M15.2_HUMAN"   "RP11-467H10.2_HUMAN" 
[157] "RP11-326I11.4_HUMAN"  "RP11-10A14.4_HUMAN"   "AC144449.1_HUMAN"    
[160] "RP11-323I15.5_HUMAN"

Add prediction stats based on highly variable genes

In [ ]:
var_genes_model = predict_day_glmnet_v2(training_mat = cardio,
                                           training_metadata = cell_metadata,
                                           holdout_mat = holdout_cardio, holdout_metadata = holdout_cell_metadata,
                                           genes.use = highly_var_genes,
                                           standardize_glmnet = FALSE,
                                           lambda = 0.000001,
                                           title = "highly_variable_gene_coeff")

In [38]:
rownames(var_genes_model) = "highly_var_genes"

In [39]:
var_genes_model = as.data.frame(var_genes_model)

In [40]:
var_genes_model$ngenes = length(highly_var_genes)
var_genes_model$gene_type = "highly_var"

In [41]:
all_stats_df = rbind(all_stats_df, var_genes_model)

In [42]:
tail(all_stats_df)

,Sensitivity,Specificity,Pos Pred Value,Neg Pred Value,Precision,Recall,F1,Prevalence,Detection Rate,Detection Prevalence,Balanced Accuracy,Accuracy,Kappa,AccuracyLower,AccuracyUpper,AccuracyNull,AccuracyPValue,McnemarPValue,ngenes,gene_type
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
lambda96,0.981289,0.9735683,0.9874477,0.9608696,0.9874477,0.981289,0.9843587,0.6793785,0.6666667,0.6751412,0.9774286,0.9788136,0.9515369,0.9652966,0.9880948,0.6793785,6.804851e-94,0.6055766,82,selection
lambda97,0.981289,0.9735683,0.9874477,0.9608696,0.9874477,0.981289,0.9843587,0.6793785,0.6666667,0.6751412,0.9774286,0.9788136,0.9515369,0.9652966,0.9880948,0.6793785,6.804851e-94,0.6055766,82,selection
lambda98,0.981289,0.9735683,0.9874477,0.9608696,0.9874477,0.981289,0.9843587,0.6793785,0.6666667,0.6751412,0.9774286,0.9788136,0.9515369,0.9652966,0.9880948,0.6793785,6.804851e-94,0.6055766,83,selection
lambda99,0.981289,0.9735683,0.9874477,0.9608696,0.9874477,0.981289,0.9843587,0.6793785,0.6666667,0.6751412,0.9774286,0.9788136,0.9515369,0.9652966,0.9880948,0.6793785,6.804851e-94,0.6055766,83,selection
lambda100,0.981289,0.9735683,0.9874477,0.9608696,0.9874477,0.981289,0.9843587,0.6793785,0.6666667,0.6751412,0.9774286,0.9788136,0.9515369,0.9652966,0.9880948,0.6793785,6.804851e-94,0.6055766,83,selection
highly_var_genes,0.991684,0.9691630,0.9855372,0.9821429,0.9855372,0.991684,0.9886010,0.6793785,0.6737288,0.6836158,0.9804235,0.9844633,0.9642115,0.9723709,0.9922193,0.6793785,1.896109e-99,0.5464936,1877,highly_var


In [43]:
write.csv(unique(all_stats_df), file = "lambda_sequence_stats_glmnet.csv")

In [46]:
write.csv(all_stats_df, file = "lambda_sequence_stats_glmnet_ALL.csv")

In [44]:
gene_set_sizes = unique(all_stats_df$ngenes)

***
### Cell age predictions accuracy in holdout cells using sets of random genes (same sizes as selected gene sets at different sparsity levels)

In [ ]:
random_list = lapply(gene_set_sizes, FUN = function(g) {

    r = lapply(1:100, FUN = function(i) {
        title = paste0("random", g, "_", i)
        genes.use = sample(training_nozero_genes, size = g, replace = FALSE)

        current_df = predict_day_glmnet_v2(training_mat = cardio,
                                           training_metadata = cell_metadata,
                                           holdout_mat = holdout_cardio,
                                           holdout_metadata = holdout_cell_metadata,
                                           genes.use = genes.use,
                                           standardize_glmnet = FALSE,
                                           lambda = 0.000001,
                                           title = title)
        rownames(current_df) = title
        current_df = as.data.frame(current_df)
        current_df$ngenes = g
        current_df$gene_type = "random"
        return(current_df)
        })
    do.call("rbind", r)
    })

random_df = do.call("rbind", random_list)

In [51]:
write.csv(random_df, file = "random_gene_stats_glmnet.csv")